In [3]:
pip install pandas_ta


     -------------------------------------- 115.1/115.1 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for pandas_ta: started
  Running setup.py install for pandas_ta: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: pandas_ta is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import pandas as pd
import pandas_ta as ta

rel_path = "../EURUSD_Candlestick_15_M_BID_01.02.2023-17.02.2024.csv"
df = pd.read_csv(rel_path)
# Removing fractions of seconds 
df["Gmt time"] = df["Gmt time"].str.replace(".000", "")
# Changing Date time format
df["Gmt time"] = pd.to_datetime(df["Gmt time"], format = '%d.%m.%Y %H:%M:%S')
#Filtering out weekends and holidays
df = df[df.High != df.Low]
df.set_index("Gmt time", inplace = True, drop = True)


In [19]:
#Adding indicators to DataFrame
df["EMA"] = ta.ema(df.Close, length = 200)
macd = ta.macd(df.Close)
df['MACD'], df['MACD_signal'], df['MACD_hist'] = macd.iloc[:,0], macd.iloc[:,1], macd.iloc[:,2]
df['ATR'] = ta.atr(df.High, df.Low, df.Close, length = 7)

df

,Open,High,Low,Close,Volume,EMA,MACD,MACD_signal,MACD_hist,ATR
Gmt time,,,,,,,,,,
2023-02-01 00:00:00,1.08605,1.08619,1.08583,1.08604,2184.41,NaN,NaN,NaN,NaN,NaN
2023-02-01 00:15:00,1.08604,1.08623,1.08583,1.08609,2373.76,NaN,NaN,NaN,NaN,NaN
2023-02-01 00:30:00,1.08608,1.08637,1.08604,1.08630,1649.21,NaN,NaN,NaN,NaN,NaN
2023-02-01 00:45:00,1.08629,1.08638,1.08606,1.08622,2071.47,NaN,NaN,NaN,NaN,NaN
2023-02-01 01:00:00,1.08622,1.08634,1.08594,1.08607,2021.66,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-02-16 20:45:00,1.07744,1.07770,1.07729,1.07764,3288.91,1.075942,0.000287,-0.000068,0.000355,0.000443
2024-02-16 21:00:00,1.07764,1.07771,1.07747,1.07763,1677.19,1.075959,0.000263,-0.000074,0.000336,0.000414
2024-02-16 21:15:00,1.07763,1.07778,1.07758,1.07776,964.40,1.075977,0.000251,-0.000068,0.000319,0.000384


In [20]:
pip install tqdm

     -------------------------------------- 78.3/78.3 kB 624.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    #Finding range of candles in window
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    #Returning Signal of current window
    if all(relevant_rows['High'] < relevant_rows['EMA']):
        return 1
    elif all(relevant_rows['Low'] > relevant_rows['EMA']):
        return 2
    else:
        return 0

df = df[-20000:-1]
from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace = True)
df['EMAsignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 5) if row.name >= 20 else 0, axis = 1)

100%|██████████| 18174/18174 [01:21<00:00, 223.79it/s]
C:\Users\krist\AppData\Local\Temp\ipykernel_28528\1238899014.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMAsignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 5) if row.name >= 20 else 0, axis = 1)


In [34]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles) == 2 and
        all(df.loc[current_candle - 3: current_candle - 2, 'MACD'] < df.loc[current_candle - 3: current_candle - 2, "MACD_signal"]) and
        all(df.loc[current_candle - 1: current_candle, 'MACD'] > df.loc[current_candle - 1: current_candle, "MACD_signal"]) 
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles) == 1 and
        all(df.loc[current_candle - 3: current_candle - 2, 'MACD'] > df.loc[current_candle - 3: current_candle - 2, "MACD_signal"]) and
        all(df.loc[current_candle - 1: current_candle, "MACD"] < df.loc[current_candle - 1: current_candle, "MACD_signal"]) 
        ):
          return 1
    return 0

df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7) if row.name != 0 else 0, axis = 1)

100%|██████████| 18174/18174 [04:48<00:00, 63.08it/s] 


In [35]:
df[df.TotalSignal != 0].head(20)


,index,Gmt time,Open,High,Low,Close,Volume,EMA,MACD,MACD_signal,MACD_hist,ATR,EMAsignal,TotalSignal
416,416,2023-02-07 08:15:00,1.07043,1.07164,1.07025,1.07107,8675.6700,1.078403,-0.000443,-0.000339,-0.000104,0.001072,1,1
963,963,2023-02-15 01:15:00,1.07409,1.07435,1.07404,1.07424,1833.2900,1.072613,0.000105,0.000074,0.000031,0.000410,2,2
1266,1266,2023-02-20 05:00:00,1.06865,1.06875,1.06836,1.06856,1908.2500,1.067656,0.000102,0.000071,0.000031,0.000417,2,2
1316,1316,2023-02-20 17:30:00,1.06866,1.06876,1.06863,1.06867,612.1600,1.068064,0.000042,0.000033,0.000009,0.000436,2,2
1416,1416,2023-02-21 18:30:00,1.06461,1.06473,1.06384,1.06419,3380.5400,1.067209,-0.000346,-0.000253,-0.000093,0.000975,1,1
1474,1474,2023-02-22 09:00:00,1.06454,1.06459,1.06354,1.06444,4431.8200,1.066370,-0.000217,-0.000149,-0.000068,0.000715,1,1
1570,1570,2023-02-23 09:00:00,1.06059,1.06062,1.06003,1.06013,4614.1400,1.063659,-0.000294,-0.000216,-0.000078,0.000713,1,1
1660,1660,2023-02-24 07:30:00,1.05903,1.05950,1.05883,1.05929,3710.0100,1.061368,-0.000137,-0.000102,-0.000035,0.000501,1,1
1743,1743,2023-02-27 04:15:00,1.05445,1.05457,1.05434,1.05441,1477.3900,1.058124,-0.000178,-0.000124,-0.000054,0.000421,1,1
1859,1859,2023-02-28 09:15:00,1.06240,1.06240,1.06160,1.06188,3462.3800,1.058698,0.000577,0.000421,0.000155,0.000918,2,2


In [36]:
import numpy as np
def pointpos(x):
    if x['TotalSignal'] == 2:
        return x['Low']-1e-3
    elif x['TotalSignal'] == 1:
        return x['High']+1e-3
    else:
        return np.nan
    
df['pointpos'] = df.apply(lambda row: pointpos(row), axis = 1)

In [50]:
pip install --upgrade nbformat

Note: you may need to restart the kernel to use updated packages.


In [51]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=1100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed